#### This script sends an authenticated HTTP/GET request to a specified URL, retrieves XML data, processes it into a Pandas DataFrame, and then prints the DataFrame. 

##### With 'Requests' we make direct requests to the API endpoints. This is a clean and efficient approach.

In [120]:
import requests
from requests_ntlm import HttpNtlmAuth
import getpass
import pandas as pd
import numpy as np
import urllib3

In [121]:
# Suppress all warnings from urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [123]:
def make_authenticated_request(base_url,report_id,username,password):
    try:
         # Construct the full URL with the dynamic report_id
        url = f"{base_url}{report_id}"

        # Create an instance of HttpNtlmAuth with username and password
        auth = HttpNtlmAuth(username, password)

        # Make an HTTP GET request with SSL verification disabled, NTLM authentication, and a timeout of 20 seconds
        response = requests.get(url, verify=False, auth=auth, timeout=30)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            xml_data = response.text
            df = pd.read_xml(xml_data)
            #print(df)
            df.to_excel(r"C:\Users\Maria.diapouli\OneDrive - OFWAT\Python\validation_tool\Model Firing Order\\" + report_id + "_Fountain_data.xlsx", sheet_name="F_Outputs", index=False)
            print("Data downloaded - Completed OK")
        elif response.status_code == 409:
            # The specific message is in the response; prompt user for company ID
            company_id = input("This report has no company, please enter a company ID: ")
            # You can use the company_id in further processing or make another request with the company_id
            print(f"Company ID entered: {company_id}")
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
            print("Response content:")
            print(response.text)
#catches any exception that inherits from RequestException. If any exception occurs during the execution of the try block (e.g., network issues, timeouts, or other HTTP-related errors, TooManyRedirects), 
#it will be caught here, and the program will print an error message indicating that an error occurred.
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [118]:
%%time 
base_url = "https://fountain01/Fountain/rest-services/report/flattable/"
#Prompt the user to enter their email address and password
report_id = input("Enter the report ID: ")
username = input("Enter your email address: ")
password = getpass.getpass("Enter your password: ")
make_authenticated_request(base_url,report_id,username,password)

Enter the report ID:  22396
Enter your email address:  maria.diapouli@ofwat.gov.uk
Enter your password:  ········


Data downloaded - Completed OK
CPU times: total: 188 ms
Wall time: 13.1 s


### Dataframe Validation 

#### When comparing two dataframes in Python to find any differences (rows with different values), we need to perform data validation to ensure a meaningful and accurate comparison. 

1. Check Column Names (Headers): Ensure that the column names in both dataframes are the same. 
2. Check Data Types: Verify that the data types of corresponding columns are the same. Mismatched data types can lead to inaccurate comparisons.
3. Check Shape of Dataframes: Ensure that the shape of the dataframes is the same, i.e., the number of rows and columns matches.
4. Remove Duplicates: Check for and remove any duplicate rows in both dataframes.
5. Sort Dataframes: Sorting dataframes can help ensure that the rows are in the same order, for accurate comparison.
6. Reset Index: Reset the index after sorting to ensure that it starts from 0.


In [4]:
def validate_and_compare_dataframes(df1, df2):
    error_messages = []
    differing_rows = None

    # Check column names
    if df1.columns.tolist() != df2.columns.tolist():
        error_messages.append("Column names in the dataframes are not identical.")
        
    # Check data types
    if df1.dtypes.to_dict() != df2.dtypes.to_dict():
        error_messages.append("Data types of columns are not identical.")
    
    # Check shape of dataframes
    if df1.shape != df2.shape:
        error_messages.append("Shapes of the dataframes do not match.")
    
    # Remove duplicates
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()

    # Sort dataframes
    df1 = df1.sort_values(by=df1.columns.tolist())
    df2 = df2.sort_values(by=df2.columns.tolist())

    # Reset index
    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)

    # Raise a single exception with all error messages and differing rows
    if error_messages or differing_rows is not None:
        raise ValueError("\n".join(error_messages), differing_rows)



### Dataframe Standardization 
#### Data standardization is the process of transforming data into a common format or structure to facilitate comparison. In the context of comparing data between two DataFrames, standardization will involve: 
1. converting data types
2. rounding numeric values
3. deleting rows to ensure consistent formatting
4. ensure column name consistent. 

In [5]:
#Testing Fountain report always request 4 items so we expect data to start from row 5
def df_fountain_standardization(df):
    # Drop the first column
    df = df.drop(df.columns[0], axis=1)
    # Get values from cells G3, G4, G5 and save them as attributes
    attributes = df.iloc[2:5, 6].tolist()
    print('Fountain Attributes from Report are: ',attributes)
    # Assign columns on row 1
    df.columns = (df.iloc[1])
    # Remove rows 
    df = df.drop([0, 1, 2, 3,4])
    
    return df

In [6]:
def df_excel_model_standardization(df):
    report_name = df.iloc[0,2]
    print('Report name is: ',report_name)
    # Assign columns on row 1
    df.columns = (df.iloc[1])
    df = df.drop([0])
    df = df.drop([1,2])
    
    return df
    

In [7]:
def compare_column_names(df1, df2):
    # Extract column names
    column_names_df1 = df1.columns.tolist()
    column_names_df2 = df2.columns.tolist()

    # Compare column names
    different_columns = set(column_names_df1) ^ set(column_names_df2)

    if different_columns:
        print("Differing column names:")
        for column in different_columns:
            if column in column_names_df1:
                print(f"Column: {column}, Values in df1: {df1[column].tolist()}")
            if column in column_names_df2:
                print(f"Column: {column}, Values in df2: {df2[column].tolist()}")
    else:
        print("*Column names are the same for both DataFrames.")

###  Dataframe Comparison. 
#### Using the merge function in pandas to identify rows that exist in one DataFrame but not in the other. The merge function is used to merge df1 and df2, the '_merge' column is added to indicate the source of each row (when value is both it means that data exist in both dataframes). The differing_rows DataFrame contains rows that are unique to either df1 or df2

In [8]:
def sort_dataframe(df, columns):
    return df.sort_values(by=columns)

In [17]:
def find_differing_rows(df1, df2):

    # Merge DataFrames and identify differing rows
    merged_df = pd.merge(df1, df2, how='outer', indicator=True)
    # Select rows that are different
    different_rows = merged_df[merged_df['_merge'] != 'both']

    # Drop the indicator column
    different_rows = different_rows.drop(columns=['_merge'])

    return different_rows

In [10]:
def convert_to_numeric(value):
    try:
        return round(pd.to_numeric(value),6)
    except (ValueError, TypeError):
        return str(value)

In [16]:
%%time 
import os
import pandas as pd

path = r"C:\Users\Maria.diapouli\OneDrive - OFWAT\Python\validation_tool\Model Firing Order\Examples"
os.chdir(path)  # Change the directory to the O drive

file_name1='22396_Fountain_data.xlsx'
file_name2='22396_FM_WWW4_Run8a.xlsx'

# Extract base name without extension
# Extract base name without extension
base_name1 = os.path.splitext(file_name1)[0]
base_name2 = os.path.splitext(file_name2)[0]

# Load data
df1 = pd.read_excel(file_name1, sheet_name='F_Outputs',header=None)
df2 = pd.read_excel(file_name2, sheet_name='F_Outputs',header=None)

df1 = df_fountain_standardization(df1)
#.sort_values(by=['Acronym', 'Reference'])
df2 = df_excel_model_standardization(df2)

############
#Standardized Column Names
# Extract column names from the first row
column_names_df1 = df1.columns.tolist()
column_names_df2 = df2.columns.tolist()

# New column names for the first 5 columns
common_columns = ['Acronym', 'Reference', 'Item description', 'Unit', 'Model']

# Assign new column names for first 5 columns of df, and renames them based on the provided mapping (common_columns). The rest of the column names beyond the first 5 columns will remain unchanged.
df1.rename(columns=dict(zip(column_names_df1[:5], common_columns)), inplace=True)
df2.rename(columns=dict(zip(column_names_df2[:5], common_columns)), inplace=True)

compare_column_names(df1, df2)

##Sort dataframes
# Specify columns for sorting
sort_columns = ['Acronym', 'Reference']
df1 = sort_dataframe(df1, sort_columns)
df2 = sort_dataframe(df2, sort_columns)

##Dataframe Validation
try:
    validate_and_compare_dataframes(df1, df2)
    print("**Dataframes Validation Completed.")
except ValueError as e:
    error_message, differing_rows = e.args
    print(f"Validation failed:\n{error_message}")

# Due to the way Fountain stores represent floating-point numbers, comparing them directly for equality may lead to unexpected results due to rounding errors.Rounding the values to 6 decimal places.
# Convert values in columns 5 and beyond to numeric
df1.iloc[:, 5:] = df1.iloc[:, 5:].applymap(convert_to_numeric)
df2.iloc[:, 5:] = df2.iloc[:, 5:].applymap(convert_to_numeric)



df1.to_excel(os.path.join(".\\Outputs\\" , base_name1 + "_Standardized.xlsx"), sheet_name="F_Outputs", index=False)
df2.to_excel(os.path.join(".\\Outputs\\" , base_name2 +"_Standardized.xlsx"), sheet_name="F_Outputs", index=False)

df_differing_rows = find_differing_rows(df1, df2)
df_differing_rows = sort_dataframe(df_differing_rows, sort_columns)
df_differing_rows.drop_duplicates()
df_differing_rows.to_excel(os.path.join(".\\Outputs\\" , "differing_rows.xlsx"), sheet_name="F_Outputs",  index=False)

Fountain Attributes from Report are:  ['Price Review 2019', 'PR19 Run 8A: 7 November draft FD', 'Latest']
Report name is:  PR19CA008_OUT2
*Column names are the same for both DataFrames.
**Dataframes Validation Completed.
CPU times: total: 438 ms
Wall time: 457 ms
